# Create Different veactor Representations of News articles 

In [1]:
import pandas as pd
import re
import numpy as np
import pickle

In [2]:
#import preprocessed data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer,LancasterStemmer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [7]:
df=pd.read_excel('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/Data_Train.xlsx')
df.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [8]:
df['SECTION'].value_counts()

1    2772
2    1924
0    1686
3    1246
Name: SECTION, dtype: int64

In [9]:
df['STORY'][0]

'But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront.\n\n\nGill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.\n\n\nBut there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor.'

In [10]:
df['STORY'][10]

'“One would think that their development and testing process would’ve uncovered many of these flaws, and yet they proceeded to put it in the market anyway," said Bryan Ma, vice president of devices research at consultancy IDC. “Clearly they can’t afford to have another embarrassing Note 7-like incident, lest they build up a reputation for releasing unreliable products."\n\n\nThe Note 7 episode triggered a global recall, cost the company billions of dollars and marred its reputation as it battled Apple Inc. in premium devices. Pulling the Fold now lets the Korean giant address potential issues as it races to put out a flexible gadget ahead of Chinese rival Huawei Technologies Co. and Xiaomi Corp.\n\n\nA Samsung spokeswoman declined to comment for this story. Shares in the company, which hasn’t set another date for a commercial launch, were little changed in Seoul on Wednesday.\n\n\nSamsung has bounced back since the Note 7 — it remains the world’s largest producer of smartphones and mem

In [11]:
df_te=pd.read_excel('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/Data_Test.xlsx')
df_te.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [12]:
y = df['SECTION'].values
X = df['STORY']
X_test=df_te['STORY']
# y_test=df_s['SECTION'].values
 
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.12, stratify=y,random_state=24) 
print(X_train.shape,y_train.shape) 
print(X_cv.shape,y_cv.shape)
print(X_test.shape)

(6712,) (6712,)
(916,) (916,)
(2748,)


In [13]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain','aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","percentage"]

In [14]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won’t", "will not", phrase)
    phrase = re.sub(r"can’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
   
    return phrase

Output from below function is encoded into TfIdf representations

In [15]:
 
stemmer=LancasterStemmer()
def preprocess_text_stemmer(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace(',', '')
        sent = re.sub('\sCo\.',' company',sent)
        sent = re.sub('\sCorp\.',' corporation',sent)
        sent = re.sub('&','and',sent)
        # sent = re.sub('\.','',sent)
        # sent = re.sub('%',' percentage ',sent)
        sent = re.sub('%',' ',sent)
        sent = re.sub('\$',' dollars ',sent)
        sent = re.sub('[^A-Za-z0-9-\.]+', ' ', sent)
        # lemmatizer = WordNetLemmatizer() 
        sent = ' '.join(stemmer.stem(e) for e in sent.split() if e.lower() not in stopwords and len(e)>1)
        #sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_text.append(sent.strip())
    return preprocessed_text

In [16]:
X_train_stemmer = preprocess_text_stemmer(X_train.values)
X_cv_stemmer = preprocess_text_stemmer(X_cv.values)
X_test_stemmer = preprocess_text_stemmer(X_test.values)
#1100,100,10

100%|██████████| 2748/2748 [00:03<00:00, 800.80it/s]


In [26]:
X_train_stemmer[0]

'airtel vodafon ide continu ced ground rely jio market shar loss two consecut quart companies. mint pac rely jio month us bas addit slow stil easy outpac airtel vodafon ide rely jio gain shar urb area despit airtel vodafon ide foc reg'

In [27]:
X_train[0:1]

1803    Airtel and Vodafone Idea continue to cede grou...
Name: STORY, dtype: object

Output from below function is encoded into word to vectors 

In [28]:
def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace(',', '')
        sent = re.sub('\sCo\.',' company',sent)
        sent = re.sub('\sCorp\.',' corporation',sent)
        sent = re.sub('&','and',sent)
        # sent = re.sub('\.','',sent)
        # sent = re.sub('%',' percentage ',sent)
        sent = re.sub('%',' ',sent)
        sent = re.sub('\$',' dollars ',sent)
        sent = re.sub('[^A-Za-z0-9-\.]+', ' ', sent)
        # lemmatizer = WordNetLemmatizer() 
        sent = ' '.join(e.lower() for e in sent.split() if e.lower() not in stopwords and len(e)>1)
        # sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_text.append(sent.strip())
  
    return preprocessed_text

In [29]:
X_train_p= preprocess_text(X_train.values)
X_cv_p= preprocess_text(X_cv.values)
X_test_p= preprocess_text(X_test.values)
#1100,100,10

100%|██████████| 2748/2748 [00:00<00:00, 3092.03it/s]


In [30]:
X_train_p[0]

'airtel vodafone idea continue cede ground reliance jio market share losses two consecutive quarters companies. mint pace reliance jio monthly user base additions slowed still easily outpacing airtel vodafone idea reliance jio gained share urban areas despite airtel vodafone idea focus regions'

Output from below function is encoded into BERT sentence vectors 

In [31]:
def preprocess_text_bert(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        # sent = sent.replace(',', '')
        sent = re.sub('\sCo\.',' company',sent)
        sent = re.sub('\sCorp\.',' corporation',sent)
        sent = re.sub('&','and',sent)
        # sent = re.sub('\.','',sent)
        sent = re.sub('%',' percentage ',sent)
        sent = re.sub('\$',' dollars ',sent)
        # lemmatizer = WordNetLemmatizer() 
        # sent = ' '.join(e.lower() for e in sent.split() if e.lower() not in stopwords and len(e)>1)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_text.append(sent.strip())
  
    return preprocessed_text

In [32]:
X_train_bert = preprocess_text_bert(X_train.values)
X_cv_bert = preprocess_text_bert(X_cv.values)
X_test_bert = preprocess_text_bert(X_test.values)

100%|██████████| 2748/2748 [00:00<00:00, 15880.90it/s]


In [33]:
X_train_bert[0]

"airtel and vodafone idea continue to cede ground to reliance jio, with market share losses in two consecutive quarters for both companies. (mint) while the pace of reliance jio is monthly user base additions has slowed, it is still easily outpacing airtel and vodafone idea reliance jio has gained share in urban areas despite airtel and vodafone idea's focus on these regions"

# Create glove model to convert text to word to vectors 

In [34]:
sentences=[]
for i in X_train_p:
  sentences.append(list(i.split()))

for i in X_cv_p:
  sentences.append(list(i.split()))

for i in X_test_p:
  sentences.append(list(i.split()))

In [35]:
from gensim.models import Word2Vec
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)

Word2Vec(vocab=51170, size=100, alpha=0.025)


In [36]:
model.vocabulary

In [37]:
print(words)
# access vector for one word
print(model['sentence'])
# save model
model.save('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/model_gensim.bin')
# load model
new_model = Word2Vec.load('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/model_gensim.bin')
print(new_model)

['airtel', 'vodafone', 'idea', 'continue', 'cede', 'ground', 'reliance', 'jio', 'market', 'share', 'losses', 'two', 'consecutive', 'quarters', 'companies.', 'mint', 'pace', 'monthly', 'user', 'base', 'additions', 'slowed', 'still', 'easily', 'outpacing', 'gained', 'urban', 'areas', 'despite', 'focus', 'regions', 'reason', 'called', 'brahmastra', 'energies', 'wisdom', 'powers', 'film', 'come', 'ancient', 'india', 'electronic', 'device', 'defined', 'claim', 'comprising', 'touch', 'sensor', 'wherein', 'front', 'face', 'rear', 'flexible', 'display', 'layer', 'configured', 'content', 'based', 'input', 'gathered', 'using', 'face.', 'may', 'mean', 'apple', 'might', 'working', 'glass-unibody', 'design', 'wrap', 'around', 'edges', 'device.', 'clicking', 'episode', 'name', 'would', 'open', 'full', 'page', 'episodes', 'showing', 'complete', 'details', 'help', 'decide', 'listen', 'report', 'said.', 'prior', 'podcasts', 'web', 'list', 'redirected', 'users', 'itunes', 'no', 'playback', 'option', 'av

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Word2Vec(vocab=51170, size=100, alpha=0.025)


In [38]:
pickle.dump((X_train_p, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_data.pkl','wb'))
pickle.dump((X_test_p),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_data.pkl','wb'))
pickle.dump((X_cv_p, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_data.pkl','wb'))

In [39]:
X_train_p,y_train = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_data.pkl", 'rb')) 
X_test_p = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_data.pkl",'rb'))
X_cv_p,y_cv =  pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_data.pkl",'rb'))

In [40]:
pickle.dump((X_train_stemmer, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_data_stemmer.pkl','wb'))
pickle.dump((X_test_stemmer),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_data_stemmer.pkl','wb'))
pickle.dump((X_cv_stemmer, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_data_stemmer.pkl','wb'))

In [41]:
pickle.dump((X_train_bert, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_data_bert.pkl','wb'))
pickle.dump((X_test_bert),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_data_bert.pkl','wb'))
pickle.dump((X_cv_bert, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_data_bert.pkl','wb'))

# Create TfIdf representation of each article

In [42]:
#TfIdf of words of essay column
tfidf1 = TfidfVectorizer(min_df=5,ngram_range=(1,4))
tfidf1.fit(X_train_stemmer)

# we use the fitted TfIdfVectoriser to convert the text to vector
X_train_tfidf = tfidf1.transform(X_train_stemmer)
X_cv_tfidf = tfidf1.transform(X_cv_stemmer)
X_test_tfidf = tfidf1.transform(X_test_stemmer)

print("After vectorizations")
print(X_train_tfidf.shape, y_train.shape)
print(X_cv_tfidf.shape, y_cv.shape)
print(X_test_tfidf.shape)

After vectorizations
(6712, 16859) (6712,)
(916, 16859) (916,)
(2748, 16859)


In [43]:
pickle.dump((X_train_tfidf, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_tfidf.pkl','wb'))
pickle.dump((X_test_tfidf),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_tfidf.pkl','wb'))
pickle.dump((X_cv_tfidf, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_tfidf.pkl','wb'))

In [44]:
X_train_tfidf,y_train = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_tfidf.pkl", 'rb')) 
X_test_tfidf = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_tfidf.pkl",'rb'))
X_cv_tfidf,y_cv =  pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_tfidf.pkl",'rb'))

In [45]:
print(X_train_tfidf.shape,y_train.shape)
print(X_test_tfidf.shape)
print(X_cv_tfidf.shape,y_cv.shape)

(6712, 16859) (6712,)
(2748, 16859)
(916, 16859) (916,)


# Create TfIdf word to vectors from model built by News article data

In [46]:
#converting to word to vector
infile = open('/content/drive/My Drive/ data_sets/glove_vectors','rb')
w2v_model = pickle.load(infile)
glove_words =  set(w2v_model.keys())

In [47]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(X_train_p)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words_essay = set(tfidf_model.get_feature_names())

In [48]:
def w2v_tfidf(X):
    tfidf_w2v = []; # the avg-w2v for each sentence/review is stored in this list
    for sentence in tqdm(X): # for each review/sentence
        vector = np.zeros(300) # as word vectors are of zero length
        tf_idf_weight =0; # num of words with a valid vector in the sentence/review
        for word in sentence.split(): # for each word in a review/sentence
            if (word in glove_words) and (word in tfidf_words_essay):
                vec = w2v_model[word] # getting the vector for each word
                # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
                tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
                vector += (vec * tf_idf) # calculating tfidf weighted w2v
                tf_idf_weight += tf_idf
        if tf_idf_weight != 0:
            vector /= tf_idf_weight
        tfidf_w2v.append(vector)
    return np.array(tfidf_w2v)

In [49]:
tfidf_w2v_tr= w2v_tfidf(X_train_p)
tfidf_w2v_cv= w2v_tfidf(X_cv_p)
tfidf_w2v_te= w2v_tfidf(X_test_p)

100%|██████████| 2748/2748 [00:01<00:00, 2008.26it/s]


In [50]:
model.wv.vocab.keys()

dict_keys(['airtel', 'vodafone', 'idea', 'continue', 'cede', 'ground', 'reliance', 'jio', 'market', 'share', 'losses', 'two', 'consecutive', 'quarters', 'companies.', 'mint', 'pace', 'monthly', 'user', 'base', 'additions', 'slowed', 'still', 'easily', 'outpacing', 'gained', 'urban', 'areas', 'despite', 'focus', 'regions', 'reason', 'called', 'brahmastra', 'energies', 'wisdom', 'powers', 'film', 'come', 'ancient', 'india', 'electronic', 'device', 'defined', 'claim', 'comprising', 'touch', 'sensor', 'wherein', 'front', 'face', 'rear', 'flexible', 'display', 'layer', 'configured', 'content', 'based', 'input', 'gathered', 'using', 'face.', 'may', 'mean', 'apple', 'might', 'working', 'glass-unibody', 'design', 'wrap', 'around', 'edges', 'device.', 'clicking', 'episode', 'name', 'would', 'open', 'full', 'page', 'episodes', 'showing', 'complete', 'details', 'help', 'decide', 'listen', 'report', 'said.', 'prior', 'podcasts', 'web', 'list', 'redirected', 'users', 'itunes', 'no', 'playback', 'op

In [51]:
def w2v_tfidf2(X):
    tfidf_w2v = []; # the tfidf-w2v for each sentence/review is stored in this list
    for sentence in tqdm(X): # for each review/sentence
        vector = np.zeros(100) # as word vectors are of zero length
        tf_idf_weight =0; # num of words with a valid vector in the sentence/review
        for word in sentence.split(): # for each word in a review/sentence
            if (word in model.wv.vocab.keys()) and (word in tfidf_words_essay):
                vec = model[word] # getting the vector for each word
                # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
                tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
                vector += (vec * tf_idf) # calculating tfidf weighted w2v
                tf_idf_weight += tf_idf
        if tf_idf_weight != 0:
            vector /= tf_idf_weight
        tfidf_w2v.append(vector)
    return np.array(tfidf_w2v)

In [52]:
tfidf_w2v_tr2= w2v_tfidf2(X_train_p)
tfidf_w2v_cv2= w2v_tfidf2(X_cv_p)
tfidf_w2v_te2= w2v_tfidf2(X_test_p)

  0%|          | 0/6712 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
100%|██████████| 2748/2748 [00:03<00:00, 762.20it/s]


In [53]:
pickle.dump((tfidf_w2v_tr, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_w2v.pkl','wb'))
pickle.dump((tfidf_w2v_te),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_w2v.pkl','wb'))
pickle.dump((tfidf_w2v_cv, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_w2v.pkl','wb'))

In [54]:
tfidf_w2v_tr,y_train = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_w2v.pkl", 'rb')) 
tfidf_w2v_te = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_w2v.pkl",'rb'))
tfidf_w2v_cv,y_cv =  pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_w2v.pkl",'rb'))

In [55]:
print(tfidf_w2v_tr.shape,y_train.shape)
print(tfidf_w2v_te.shape)
print(tfidf_w2v_cv.shape,y_cv.shape)

(6712, 300) (6712,)
(2748, 300)
(916, 300) (916,)


# Create TfIdf word to vectors from gloe.6B.300.txt file 

In [56]:
embeddings_dict = {}
with  open('/content/drive/My Drive/ Deep learning assignments/ LSTM Donors_choose/glove.6B.300d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [57]:
def w2v_tfidf_300(X):
    tfidf_w2v = []; # the tfidf-w2v for each sentence/review is stored in this list
    for sentence in tqdm(X): # for each review/sentence
        vector = np.zeros(300) # as word vectors are of zero length
        tf_idf_weight =0; # num of words with a valid vector in the sentence/review
        for word in sentence.split(): # for each word in a review/sentence
            if (word in embeddings_dict.keys()) and (word in tfidf_words_essay):
                vec = embeddings_dict[word] # getting the vector for each word
                # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
                tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
                vector += (vec * tf_idf) # calculating tfidf weighted w2v
                tf_idf_weight += tf_idf
        if tf_idf_weight != 0:
            vector /= tf_idf_weight
        tfidf_w2v.append(vector)
    return np.array(tfidf_w2v)

In [58]:
tfidf_300_tr= w2v_tfidf_300(X_train_p)
tfidf_300_cv= w2v_tfidf_300(X_cv_p)
tfidf_300_te= w2v_tfidf_300(X_test_p)

100%|██████████| 2748/2748 [00:02<00:00, 1049.19it/s]


In [59]:
pickle.dump((tfidf_w2v_tr, y_train),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_300.pkl','wb'))
pickle.dump((tfidf_w2v_te),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_300.pkl','wb'))
pickle.dump((tfidf_w2v_cv, y_cv),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_300.pkl','wb'))

In [60]:
tfidf_300_tr,y_train = pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/train_300.pkl", 'rb')) 
tfidf_300_te= pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/test_300.pkl",'rb'))
tfidf_300_cv,y_cv =  pickle.load(open("/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/cv_300.pkl",'rb'))

In [61]:
print(tfidf_300_tr.shape,y_train.shape)
print(tfidf_300_te.shape)
print(tfidf_300_cv.shape,y_cv.shape)

(6712, 300) (6712,)
(2748, 300)
(916, 300) (916,)


# Create each word to tokens

In [62]:
t = Tokenizer()
t.fit_on_texts(X_train_stemmer)
vocab_size_text = len(t.word_index) + 1
# integer encode the documents
X_train_tokens = t.texts_to_sequences(X_train_stemmer)
X_test_tokens = t.texts_to_sequences(X_test_stemmer)
X_cv_tokens = t.texts_to_sequences(X_cv_stemmer)

# pad documents to a max length
max_len=max([len(s) for s in X_train_tokens])
# max_len_text=int(np.percentile([len(s) for s in X_train_toke],96))
X_train_tokens = pad_sequences(X_train_tokens, maxlen=max_len, padding='post')
X_test_tokens = pad_sequences(X_test_tokens, maxlen=max_len, padding='post')
X_cv_tokens = pad_sequences(X_cv_tokens, maxlen=max_len, padding='post')

print('shape of X_train_tokens ',X_train_tokens.shape)
print('shape of X_test_tokens ',X_test_tokens.shape)
print('shape of X_cv_tokens ',X_cv_tokens.shape)

shape of X_train_tokens  (6712, 591)
shape of X_test_tokens  (2748, 591)
shape of X_cv_tokens  (916, 591)


In [63]:
pickle.dump((X_train_tokens,X_test_tokens,X_cv_tokens,max_len,vocab_size_text),open('/content/drive/MyDrive/ Competitions/ Predict_the_news_category/ data/tokens.pkl','wb'))

In [64]:
print(X_train_tokens.shape)
print(X_test_tokens.shape)
print(X_cv_tokens.shape)
print(max_len,vocab_size_text)

(6712, 591)
(2748, 591)
(916, 591)
591 24749
